In [4]:
import cftime
import glob
import netCDF4 as nc
import numpy as np
import pandas as pd

import mission_tools

%load_ext autoreload
%autoreload 2
%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
mission_list = glob.glob('../microSWIFT_data/final_dataset/mission_*.nc')
bathy_file = '../microSWIFT_data/FRFdata/' \
             'FRF_geomorphology_DEMs_surveyDEM_20211021.nc'
awac_file = '../microSWIFT_data/FRFdata/FRF-ocean_waves_awac-4.5m_202110.nc'

In [ ]:
microSWIFT_sig_wave_heights = []
mission_times = []
standard_deviations = []
awac_sig_wave_heights = []
num_waves_in_mission = []
mean_wave_periods  = []

for mission_nc in mission_list:
    mission_dataset = nc.Dataset(mission_nc, mode='r')

    x_locations = mission_dataset['xFRF'][:]
    y_locations = mission_dataset['yFRF'][:]
    eta = mission_dataset['sea_surface_elevation'][:]
    time = mission_dataset['time'][:]

    wave_heights, \
    wave_x_locs, \
    wave_y_locs, \
    wave_periods = mission_tools.compute_individual_waves(x_locations,
                                                          y_locations,
                                                          eta, time,
                                                          bathy_file)

    sig_wave_height, \
    standard_dev  = mission_tools.compute_sig_wave_height_top_third(wave_heights)

    microSWIFT_sig_wave_heights.append(sig_wave_height)
    num_waves_in_mission.append(len(wave_heights))
    standard_deviations.append(standard_dev)
    mission_time = np.median(mission_dataset['time'][:])
    mission_times.append(mission_time)
    mean_wave_periods.append(np.mean(wave_periods))

    awac_sig_wave_heights.append(mission_tools.closest_awac_sig_wave_height(mission_time, awac_file))